In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../autoencoder/")
from model import MoleculeVAE

Using Theano backend.


In [3]:
import pandas
import numpy as np
data = pandas.read_hdf("test",'table')
charset = list(np.load("symbols.npy"))
print(charset)

['m', '5', ':', 'W', '4', '7', 's', 'I', 'O', '\\', '[', 'f', 'H', '9', 'b', 'r', 'K', 'n', 'd', '8', 'T', '+', 'S', 'F', 'c', '(', 'Y', 'e', 'R', 'P', 'a', 'V', 'M', 'D', 'N', '0', 'Z', 'X', 'p', 'l', '.', '2', 'E', 'G', 'y', '#', '-', '6', '1', 'A', 'g', '/', '%', ')', ']', '3', 'B', '@', 'i', 'L', 'o', 'u', '=', 'h', 'C', 't']


In [4]:
from sklearn.cross_validation import train_test_split
import functools
import numpy as np
import sys
sys.path.append("../")
from autoencoder.utils import one_hot_array, one_hot_index

smiles_column = 'structure'
keys = data[smiles_column].map(len) < 121

#length = len(keys) - 20
#print(len(data[keys]),length)
#length = 10

#if length <= len(keys):
#    data = data[keys].sample(n = length)
#else:
    
data = data[keys]

structures = data[smiles_column].map(lambda x: list(x.ljust(120)))

#if args.property_column:
#    properties = data[args.property_column][keys]

#del data

train_idx, test_idx = train_test_split(range(structures.shape[0]), test_size = 0.20)

charset = list(functools.reduce(lambda x, y: set(y) | x, structures, set()))

In [5]:
print(charset)
print (structures.shape)

['f', 'E', ')', '6', '%', 'C', 'O', 'D', 'K', 't', 'G', 'n', 'T', '[', '-', 'M', 'I', '/', 'N', 'Z', 'u', 'S', 'p', 'g', '(', 'B', 'Y', 'H', '0', 'i', 'y', ':', 'o', '@', '8', 'X', 'A', '2', 'e', ']', 'a', '1', 'L', 'd', ' ', '7', 'P', 'R', 'm', 'c', '\\', '=', 'b', 's', 'F', '3', 'l', 'V', 'r', '9', '#', '4', '+', '5']
(1001433,)


In [27]:
batch_size = 10
def one_hot_encoded(sub_structures):
    return  np.array(list(
    map(lambda row:
        list(map(lambda x: one_hot_array(x, len(charset)),
            one_hot_index(row, charset))),
        sub_structures)))

def generator(test=False):
    size = batch_size
    if test:
        idx = test_idx
    else:
        idx = train_idx
    i = 0
    while True:
        sample = idx[i * size:(i + 1) * size]
        #print(structures[sample])
        if (i + 1) * size > len(idx):
            i = 0
        else:
            i += 1
        X = one_hot_encoded(structures[sample])
        yield (X,X)

In [7]:
#one_hot_encoded(structures[[0,1]]).shape

In [28]:
train = generator()
for i in train:
    print(i[0].shape)
    break
test = generator(test=True)

(10, 120, 64)


In [ ]:
import os
from model import MoleculeVAE
from keras.callbacks2 import ModelCheckpoint, ReduceLROnPlateau
model = MoleculeVAE()
mfile ="new.h5"
latent_dim = 292

In [22]:
int(len(train_idx) / batch_size)

1335

In [ ]:
if os.path.isfile(mfile):
    model.load(charset, mfile, latent_rep_size = latent_dim)
else:
    model.create(charset, latent_rep_size = latent_dim)

checkpointer = ModelCheckpoint(filepath = mfile,
                               verbose = 1,
                               save_best_only = True,
                              save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 3,
                              min_lr = 0.0001)

In [ ]:
model.autoencoder.fit_generator(train,
                                samples_per_epoch = int(len(train_idx) / batch_size) * batch_size,
                                nb_epoch=20,
                                validation_data = test,
                                nb_val_samples = 2,
                                callbacks = [checkpointer, reduce_lr])

Epoch 1/20
    50/801140 [..............................] - ETA: 1533931s - loss: 3.9808 - acc: 0.6393